In [1]:
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.models import TCNModel
from darts.metrics import mape, rmse
from darts.utils.timeseries_generation import datetime_attribute_timeseries
import pandas as pd

# Giả sử dữ liệu của bạn được lưu trữ trong một DataFrame có tên là df
# với 12 cột biến đầu vào (VAR78, VAR79, ..., VAR228) và 1 cột đầu ra (solar energy)
train_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks (1)/GEFCom2014-Solar-Energy-Forecasting/GEFCom2014-S_V2/Solar/Task 1/predictors1.csv", index_col='TIMESTAMP', parse_dates=True)
train_df.index = pd.to_datetime(train_df.index)
label_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks (1)/GEFCom2014-Solar-Energy-Forecasting/GEFCom2014-S_V2/Solar/Task 1/train1.csv")
# prompt: viết mã để gộp 2 dataframe trongg pandas tên là "train_df" và "label_df"
df = pd.merge(train_df, label_df)
df

ModuleNotFoundError: No module named 'darts'

In [ ]:


# Chuyển đổi DataFrame thành TimeSeries
series = TimeSeries.from_dataframe(df, time_col='TIMESTAMP', value_cols=['VAR78', 'VAR79', 'VAR134', 'VAR157', 'VAR164', 'VAR165', 'VAR166', 'VAR167', 'VAR169', 'VAR175', 'VAR178', 'VAR228', 'POWER'])

# Thêm các thuộc tính ngày giờ để giúp mô hình nắm bắt các mẫu theo mùa
series = datetime_attribute_timeseries(series, attribute="month", one_hot=True)

# Chia dữ liệu thành tập huấn luyện và tập xác thực
train, val = series.split_after(pd.Timestamp('2013-06-06'))  # Điều chỉnh ngày chia theo dữ liệu của bạn

# Chuẩn hóa dữ liệu
scaler = Scaler()
train_transformed = scaler.fit_transform(train)
val_transformed = scaler.transform(val)

# Xây dựng mô hình TCN với các siêu tham số được đề xuất
model = TCNModel(
    input_chunk_length=24,  # Độ dài của chuỗi đầu vào (tùy chỉnh theo độ dài phụ thuộc)
    output_chunk_length=1,  # Dự đoán 1 bước thời gian
    num_filters=32,  # Số lượng bộ lọc trong mỗi lớp tích chập
    kernel_size=3,  # Kích thước kernel
    dilation_base=2,  # Cơ số giãn nở
    num_layers=4,  # Số lớp (tùy chỉnh theo input_chunk_length và dilation_base)
    dropout=0.2,  # Tỷ lệ dropout
    random_state=42,  # Để tái lập kết quả
)

# Huấn luyện mô hình
model.fit(train_transformed, verbose=True)

# Đánh giá mô hình trên tập xác thực
pred = model.predict(n=len(val), series=val_transformed)
print("MAPE:", mape(val, pred))
print("RMSE:", rmse(val, pred))
